# Zadání - Řešte numericky rovnici advekce
## Exaktní řešení
$u(x,t) = u_0(x-vt)$
## Počáteční profil
$x\:\epsilon <0,2>$:

pro $x < 0.5$ a $x > 1$: $y = 0$

pro $x > 0.5$ a $x < 1$: $y = 1$

$v = 0.2$

$t_{end} = 2.5$

$N = 100$

$dt = 0.01$

##Okrajové podmínky
$u_1 = u_n$

## Postup
 1) Diskretizace $u(x,t)$
 2) Počáteční profil
 3) FDT (dopředná časová diference), BDS (zpětná prostorová diference)
 4) Schéma:



In [1]:
import numpy as np

## Diskretizace, počáteční hodnoty a počáteční profil

In [33]:
v = 0.2
t_end = 2.5
dt = 0.01
t_kroku = t_end / dt
N = 100

#Diskretizace prostorové souřadnice
x = np.linspace(0,2,N)
dx = np.copy(x[1])

#Počáteční profil
u = np.oneslike(x)

#u[np.where((x > 0.5) and (x < 1))] = 2.0 #Viktorova rada
x_mask = np.where([(el > 0.5) and (el < 1) for el in x])
u[x_mask] = 2.0
#print(u)

## FDT - Dopředná časová diference a BDS - Zpětná prostorová diference

In [ ]:
def FDT_BDS(u,x)
    t = 0.0
    u_new = np.oneslike(u)

    while (t < t_end):
        #Okrajová podmínka
        u_new[0] = u_new[-1]
        #Rovnice pro dopřednou časovou diferenci
        u_new[1:] = u[1:] - v*(dt/dx)(u[1:]-u[:-1])
        u = u_new
        #Uložení u_new
        #Připočtení časového kroku
        t += dt
    return u

solution = FDT_BDS(u,x)
print(solution)
